In [1]:
from components import Components
from llama_index.core import Settings

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate,Document 


from llama_index.core.node_parser import SentenceSplitter

import pandas as pd

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#initialise components, reranker and retriever
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","mixtral-8x7b-32768")

Setting Contexts


In [3]:
Settings.embed_model = rag_components.get_embedding_model()
Settings.llm = rag_components.get_groq_llm()

Embedding model loaded!


In [4]:
vector_store = ChromaVectorStore(chroma_collection=rag_components.get_db())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Loading Papers

In [5]:
df = pd.read_parquet('data/test/test_neural_bridge_qa.parquet')

In [6]:
context_list = []
for contexts in df['context'][:128]:
    context_list.append(Document(text=contexts))

In [7]:
pipeline = IngestionPipeline(
    transformations=[
          #SentenceSplitter(chunk_size=1000, chunk_overlap=200),
    ],
    vector_store=vector_store,
)

In [8]:
documents = pipeline.run(documents=context_list)

In [9]:
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context,similarity_top_k=10, node_postprocessors=[rag_components.get_reranker()]
) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mixedbread-ai/mxbai-embed-large-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Reranker model loaded!


In [10]:
query_engine = index.as_query_engine(streaming=True,similarity_top_k=10)

qa_prompt_template = PromptTemplate(rag_components.get_qa_template())
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":qa_prompt_template}
)

In [11]:
question_list = []
for questions in df['question'][:128]:
    question_list.append(questions)
    

In [13]:
response = query_engine.query(question_list[0])
response.print_response_stream()

APIConnectionError: Connection error.